In [31]:
import pandas as pd
import numpy as np


def load_data(url):
    return pd.read_csv(url)


def clean_tiobe_top_20(df):
    return df.rename(columns=lambda col: col.replace("Sept", "Rank"))

def clean_tiobe_history(df):
    return (
        df
        .drop(columns=['-'])
        .set_index('Programming Language')
        .transpose()
        .rename_axis('', axis='columns')
        .reset_index()
        .rename(columns={'index': 'year'})
        .melt(id_vars='year')
        .rename(columns={'value': 'rank', '': 'language'})
        .sort_values(by=['year', 'rank'])
        .assign(rank=lambda df: df['rank'].replace({'-': np.nan}).astype('float'))
        .query("~language.isin(['Pascal', 'Visual Basic', '(Visual) Basic', 'Prolog'])", engine='python')
    )


tiobe_top_20_url = "https://raw.githubusercontent.com/mwiemers/python_presessional_app/main/tiobe_top20.csv"
tiobe_history_url = "https://raw.githubusercontent.com/mwiemers/python_presessional_app/main/tiobe_history.csv"
gapminder_url = "https://raw.githubusercontent.com/mwiemers/python_presessional_app/main/gapminder.csv"

tiobe_top_20 = clean_tiobe_top_20(load_data(tiobe_top_20_url))
tiobe_history = clean_tiobe_history(load_data(tiobe_history_url))
gap = load_data(gapminder_url)

TypeError: load_data() missing 1 required positional argument: 'sep'

In [29]:
df1

,Sep 2024,Sep 2023,Programming Language,Ratings,Change
1,1,-,Python,20.17%,+6.01%
2,3,^,C++,10.75%,+0.09%
3,4,^,Java,9.45%,-0.04%
4,2,v,C,8.89%,-2.38%
5,5,-,C#,6.08%,-1.22%
6,6,-,JavaScript,3.92%,+0.62%
7,7,-,Visual Basic,2.70%,+0.48%
8,12,^^,Go,2.35%,+1.16%
9,10,^,SQL,1.94%,+0.50%
10,11,^,Fortran,1.78%,+0.49%


In [13]:
df = clean_tiobe_history(df)


In [14]:
df.query("language == 'SQL'")

,year,language,rank
80,1987,SQL,NaN
79,1992,SQL,NaN
78,1997,SQL,NaN
77,2002,SQL,7.0
76,2007,SQL,NaN
75,2012,SQL,NaN
74,2017,SQL,NaN
73,2022,SQL,9.0
72,2024,SQL,8.0


In [42]:
import yfinance as yf

adj_close = yf.download(["TSLA"], "2020-01-01", "2022-01-01")['Adj Close']
returns = (adj_close.pct_change()+1).cumprod()*100 - 100
returns.rename("")

returns

[*********************100%***********************]  1 of 1 completed


Date
2020-01-02            NaN
2020-01-03       2.963326
2020-01-06       4.945847
2020-01-07       9.017801
2020-01-08      14.382003
                 ...     
2021-12-27    1171.254592
2021-12-28    1164.897969
2021-12-29    1162.248375
2021-12-30    1143.829308
2021-12-31    1128.071432
Name: Adj Close, Length: 505, dtype: float64

In [43]:
%%timeit

adj_close = yf.download(["TSLA"], "2020-01-01", "2022-01-01")['Adj Close']
returns = (adj_close.pct_change()+1).cumprod()*100 - 100
returns.rename("")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

29.3 ms ± 3.03 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [44]:
%%timeit 

    returns = (
        yf.download(["TSLA"], "2020-01-01", "2022-01-01")['Adj Close']
        .pct_change()
        .apply(lambda x: x + 1)
        .cumprod()
        .apply(lambda x: x*100 -100)
        .rename("")
    )
    

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

29 ms ± 2.58 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
